In [6]:
import pickle
import pandas
import numpy as np
from datetime import datetime

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

with open('BTC-USD-60.pkl', 'rb') as f:
    data = pickle.load(f)

#:chunk = filter_df(df_chunk, event_type='Fill')
data = data.sort_values(by=['time'])



In [7]:
# ## This block process the date/time info, currently not used.
# time = np.array(data.time)
# #print(time)
# new_time = []
# year,month,day,hour,minute,second = [],[],[],[],[],[]
# for i in range(len(time)):
#     new_time.append(datetime.fromtimestamp(time[i]).strftime('%Y-%m-%d %H:%M:%S'))
#     year.append(int(new_time[i][:4]))
#     month.append(int(new_time[i][5:7]))
#     day.append(int(new_time[i][8:10]))
#     hour.append(int(new_time[i][11:13]))
#     minute.append(int(new_time[i][14:16]))
#     #second.append(int(new_time[i][17:19]))
    
# print(year[3],month[33],day[333],hour[3333],minute[10000])


In [8]:
#print(data)
#data.drop([columns = 'time'])
data = data[data.close > 200]
time = np.array(data.time)
data = data.drop('time', 1)
# data.low = np.log(data.low)
# data.high = np.log(data.high)
# data.open = np.log(data.open)
# data.close = np.log(data.close)

#print(data)

In [9]:
# mean = data.mean(axis = 0)
# std = data.std(axis = 0)
# data = (data - mean)/std

# year = np.array(year)
# month = np.array(month)
# day = np.array(day)
# hour = np.array(hour)
# minute = np.array(minute)
# #second = np.array(second)

# year = (year - year.mean())/year.std()
# month = (month - month.mean())/month.std()
# day = (day - day.mean())/day.std()
# hour = (hour - hour.mean())/hour.std()
# minute = (minute - minute.mean())/minute.std()
# second = (second - second.mean())/second.std()
# print(data)

In [10]:
def normalize_windows(window_data):
    #normalized_data = []
    #print(len(window_data))
    window_data = (window_data / window_data[0]) - 1 
#     window0 = window_data[0]
#     for i in range(0,len(window_data)):
#         #print(window0)
#         window_data[i] = (float(window_data[i]) / float(window0)) - 1
    return window_data

In [73]:
def baseline_preprocess(data, time):
    window_length = 60
    A = np.array(data.open)
    B = np.array(data.close)
    C = (A + B) / 2

    i = 0
    train_X = []
    train_Y = []
    while i < len(C) - window_length:
        if time[i+window_length] - time[i] == window_length*60: 
#             normalized_temp_C = normalize_windows(np.copy(C[i:i+window_length+1]))
#             temp = normalized_temp_C[:-1]
            temp = C[i:i+window_length]
            
            
            #print(temp[60])
            train_X.append(temp)
            #train_Y.append(normalized_temp_C[-1])
            train_Y.append(C[i+window_length])
            i = i + 1
        else:
            i = i + 60
        
    #train_X = np.transpose(train_X)
    
    normalized_train_X = []
    
    return train_X, train_Y

In [74]:
train_X, train_Y = baseline_preprocess(data, time)
np.shape(train_X)
# ss = StandardScaler()
# train_X = ss.fit_transform(train_X)

np.savetxt("train_X_normalized_baseline_regression.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_baseline_regression.csv", train_Y, delimiter = ",")

In [75]:
np.shape(train_X)

(778302, 60)

In [ ]:
def fc_preprocess(data, time, year = None, hour = None):
    A = np.array(data.open)
    B = np.array(data.close)
    C = (A + B) / 2

    i = 0
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    while i < len(C) - 60:
        if time[i+60] - time[i] == 3600:
            temp = C[i:i+60]
#         temp_year = year[i:i+60]
#         temp_hour = hour[i:i+60]

            if i % 5 == 0:
                test_X.append(temp)
                test_Y.append(C[i + 60])
            else:
                train_X.append(temp)
                train_Y.append(C[i + 60])
                

            i = i + 1
            
        else:
            i = i + 60
    
    return train_X, train_Y, test_X, test_Y

In [ ]:
train_X, train_Y, test_X, test_Y = fc_preprocess(data, time)
#train_Y = np.log(train_Y)
np.savetxt("train_X_normalized_fc_regression.csv", train_X, delimiter = ",")
np.savetxt("train_Y_normalized_fc_regression.csv", train_Y, delimiter = ",")

In [21]:
#data = data.drop(columns = ['time'])
def lstm_preprocess(data, time, year = None, hour = None):
    window_length = 60
    A = np.array(data.low)
    B = np.array(data.high)
    C = np.array(data.open)
    D = np.array(data.close)
    E = np.array(data.volume)
    i = 0
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    de_norm_train = []
    de_norm_test = []
    unnormalized_Y_train = []
    unnormalized_Y_test = []
    counter = 0
    while i < len(C) - window_length:
        if time[i+window_length] - time[i] == 60 * window_length:
            temp = []
            
            copy_A = np.copy(A[i:i+window_length+1])
            copy_B = np.copy(B[i:i+window_length+1])
            copy_C = np.copy(C[i:i+window_length+1])
            copy_D = np.copy(D[i:i+window_length+1])
            copy_E = np.copy(E[i:i+window_length+1])
            
            
            normalized_temp_A = normalize_windows(copy_A)
            normalized_temp_B = normalize_windows(copy_A)
            normalized_temp_C = normalize_windows(copy_A)
            normalized_temp_D = normalize_windows(copy_A)
            normalized_temp_E = normalize_windows(copy_A)
            #print(len(normalized_temp_D[:-1]))
            temp.append(normalized_temp_A[:-1])
            temp.append(normalized_temp_B[:-1])
            temp.append(normalized_temp_C[:-1])
            temp.append(normalized_temp_D[:-1])
            temp.append(normalized_temp_E[:-1])
#         temp.append(year[i:i+60])
#         temp.append(hour[i:i+60])
            if i % 5 == 0:
                test_X.append(temp)
                test_Y.append(normalized_temp_D[-1])
                unnormalized_Y_test.append(copy_D[-1])
                de_norm_test.append([copy_A[0], copy_B[0], copy_C[0], copy_D[0], copy_E[0]])
            else:
                train_X.append(temp)
                train_Y.append(normalized_temp_D[-1])
                unnormalized_Y_train.append(copy_D[-1])
                de_norm_train.append([copy_A[0], copy_B[0], copy_C[0], copy_D[0], copy_E[0]])
            i = i + 1
            
# Taiming            
#             counter = counter + 1
#             if counter % 100 == 0:
#                 print(counter)
        else:
            i = i + 60
    return train_X, train_Y, test_X, test_Y, de_norm_train, unnormalized_Y_train, de_norm_test, unnormalized_Y_test

In [22]:
X, Y, test_X, test_Y, de_norm_train, unnormalized_Y_train, de_norm_test, unnormalized_Y_test = lstm_preprocess(data, time)
X = np.transpose(X, (0,2,1))
test_X = np.transpose(test_X, (0,2,1))

## Taiming
# split = round(X.shape[0]*0.9)
# train_X = X[:split]
# train_Y = Y[:split]
# test_X = X[split:]
# test_Y = Y[split:]
# # for i in range(train_X.shape[2]):
# #     ss = StandardScaler()
# #     train_X[:,:,i] = ss.fit_transform(train_X[:,:,i])
# # for i in range(test_X.shape[2]):
# #     ss = StandardScaler()
# #     test_X[:,:,i] = ss.fit_transform(test_X[:,:,i])
# #train_Y = np.log(train_Y)
# print(train_X.shape)
# print(test_X.shape)
# de_norm = np.array(de_norm)
# de_norm = de_norm[split:]
# unnormalized_Y = np.array(unnormalized_Y)
# unnormalized_Y = unnormalized_Y[split:]
# print(de_norm.shape)
# print(unnormalized_Y.shape)

In [24]:
np.save("train_X_normalized_lstm_regression_v2.pkl", X)
np.save("train_Y_normalized_lstm_regression_v2.pkl", Y)
np.save("test_X_normalized_lstm_regression_v2.pkl", test_X)
np.save("test_Y_normalized_lstm_regression_v2.pkl", test_Y)
np.save("de_norm_train_v2.pkl", de_norm_train)
np.save("de_norm_test_v2.pkl", de_norm_test)
np.save("unnormalized_Y_train_v2.pkl", unnormalized_Y_train)
np.save("unnormalized_Y_test_v2.pkl", unnormalized_Y_test)

In [16]:
train_Y

[0.0016098985763897122,
 0.001379913065477023,
 0.0016323340077248893,
 0.0003904814406467949,
 0.0001377031120901595,
 -0.0004819719538224909,
 0.00020655940878122614,
 0.002554660529344055,
 0.0035446301155457327,
 0.0025548957326335664,
 0.002715765247410795,
 0.002761795166858372,
 0.002761795166858372,
 0.003200405231165826,
 0.00336211859527924,
 0.003431440283726994,
 0.0028540520634334765,
 0.0030849275962887557,
 0.0029695448999791374,
 0.0029465251720723895,
 0.0027387802071345835,
 0.002761795166858372,
 0.0026697353279632186,
 0.002853855005753747,
 0.0034766191605462016,
 0.0030149597238204873,
 0.0030379746835442756,
 0.0031990794016110158,
 0.0036363636363636598,
 0.0037054085155350247,
 0.004096662830840092,
 0.004488433651743451,
 0.004602991944764101,
 0.005063873863505597,
 0.005017261219792957,
 0.004441887226697361,
 0.004672036823935466,
 0.004672036823935466,
 0.004464902186421149,
 0.004510932105868726,
 0.004510932105868726,
 0.004487917146144937,
 0.0045340514